## Know your master 비교실험 Train code

Import

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import top_k_accuracy_score

Data load

In [3]:
df = pd.read_csv('./merge_data.csv', encoding='cp949')
df = df.sample(frac=1, random_state=7777).reset_index(drop=True)

In [4]:
df

,Unnamed: 0,timestamp_rounded,RearHeatPower266,DI_accelPedalPos,BattVoltage132,VCFRONT_chillerExvFlowm3,BMS_maxDischargePower,DI_regenLight,DI_vehicleSpeed,DI_regenLight.1,...,DI_regenLight.2_median,SteeringSpeed129_mean,SteeringSpeed129_std,SteeringSpeed129_median,DI_vehicleSpeed.1_mean,DI_vehicleSpeed.1_std,DI_vehicleSpeed.1_median,course,round,label
0,628,2023-08-18 13:26,0.409091,0.088435,0.356873,0.213333,0.009524,1,0.736111,1,...,0.0,0.185750,0.302687,0.007998,0.468630,0.346464,0.465535,2,1,6
1,714,2023-08-19 12:22,0.238095,0.460870,0.274510,0.047619,0.658156,0,0.338672,0,...,0.0,0.188079,0.321934,0.015872,0.467463,0.292303,0.436063,2,2,6
2,115,2023-08-14 12:03,0.355932,0.000000,0.133337,0.031621,0.448980,0,0.864858,0,...,0.0,0.201322,0.320330,0.015873,0.562356,0.315818,0.600904,1,1,7
3,360,2023-09-05 11:24,0.083333,0.141667,0.047076,0.000000,0.765306,0,0.538007,0,...,0.0,0.198197,0.327766,0.015934,0.506827,0.321552,0.504366,2,3,0
4,447,2023-08-20 11:54,0.275000,0.533333,0.728335,0.074074,0.076532,0,0.199845,0,...,0.0,0.176766,0.311989,0.000062,0.474193,0.285063,0.434160,0,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71145,391,2023-08-19 15:31,0.359375,0.113095,0.431392,0.000000,0.979591,1,0.537609,1,...,0.0,0.185179,0.335577,0.015873,0.474120,0.320873,0.436888,2,1,4
71146,396,2023-09-12 15:42,0.648148,0.000000,0.745102,1.000000,0.055556,1,0.862087,1,...,0.0,0.041794,0.150792,0.000000,0.426700,0.306310,0.334065,0,3,2
71147,175,2023-09-22 16:07,0.151515,0.432432,0.795276,0.900000,0.725490,0,0.995085,0,...,0.0,0.202425,0.336263,0.015934,0.505695,0.295260,0.468184,1,4,1
71148,528,2023-09-22 16:25,0.315789,0.051613,0.011825,0.640000,0.000000,0,0.075832,0,...,0.0,0.196295,0.296266,0.015873,0.536824,0.321771,0.537271,2,2,1


코스별 교차검증

valid: A코스

In [5]:
df_train = df[df['course'] != 0]
df_valid = df[df['course'] == 0]

In [6]:
# 이후의 데이터 처리 코드
train_data = df_train.drop(['Unnamed: 0','course', 'round', 'timestamp_rounded'], axis=1)
valid_data = df_valid.drop(['Unnamed: 0','course', 'round', 'timestamp_rounded'], axis=1)

In [7]:
train_data.reset_index(inplace=True, drop=True)
valid_data.reset_index(inplace=True, drop=True)

In [8]:
valid_data

,RearHeatPower266,DI_accelPedalPos,BattVoltage132,VCFRONT_chillerExvFlowm3,BMS_maxDischargePower,DI_regenLight,DI_vehicleSpeed,DI_regenLight.1,DIR_torqueActual,RearTorque1D8,...,DI_regenLight.2_mean,DI_regenLight.2_std,DI_regenLight.2_median,SteeringSpeed129_mean,SteeringSpeed129_std,SteeringSpeed129_median,DI_vehicleSpeed.1_mean,DI_vehicleSpeed.1_std,DI_vehicleSpeed.1_median,label
0,0.275000,0.533333,0.728335,0.074074,0.076532,0,0.199845,0,0.094947,0.105805,...,0.183333,0.390205,0.0,0.176766,0.311989,0.000062,0.474193,0.285063,0.434160,4
1,0.180000,0.198413,0.101975,0.073171,0.011364,0,0.400932,0,0.566121,0.929464,...,0.166667,0.375823,0.0,0.239991,0.372777,0.023870,0.520129,0.294561,0.532246,4
2,0.084746,0.619403,0.572540,0.258065,0.010204,0,0.728822,0,0.138265,0.019593,...,0.050000,0.219784,0.0,0.147070,0.270082,0.000000,0.536958,0.298795,0.602789,3
3,0.567164,0.040541,0.501991,0.056250,0.413274,1,0.666925,1,0.633269,0.723525,...,0.333333,0.475383,0.0,0.307649,0.380890,0.063488,0.505728,0.314441,0.467934,9
4,0.090909,0.341085,0.141163,0.666667,0.789474,0,0.466494,0,0.066815,0.337007,...,0.400000,0.494032,0.0,0.135240,0.284118,0.000062,0.519431,0.289234,0.533247,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24769,0.101266,0.206452,0.925196,0.074074,0.433682,0,0.067976,0,0.709606,0.952976,...,0.333333,0.475383,0.0,0.215917,0.323074,0.047678,0.426510,0.313758,0.398811,4
24770,0.211268,0.352564,0.152908,0.000000,0.000000,0,0.863812,0,0.001941,0.074455,...,0.150000,0.360085,0.0,0.238404,0.347404,0.015965,0.519813,0.304084,0.531468,9
24771,0.000000,0.000000,0.278436,0.016393,0.086736,1,0.264805,1,0.000000,0.000000,...,0.416667,0.497167,0.0,0.166399,0.330827,0.000000,0.396117,0.239205,0.264805,5
24772,0.648148,0.000000,0.745102,1.000000,0.055556,1,0.862087,1,0.207479,0.713936,...,0.233333,0.426522,0.0,0.041794,0.150792,0.000000,0.426700,0.306310,0.334065,2


In [9]:
X_train = train_data.drop('label', axis=1)
y_train = train_data['label']

In [10]:
X_train

,RearHeatPower266,DI_accelPedalPos,BattVoltage132,VCFRONT_chillerExvFlowm3,BMS_maxDischargePower,DI_regenLight,DI_vehicleSpeed,DI_regenLight.1,DIR_torqueActual,RearTorque1D8,...,VCFRONT_chillerExvFlowm3.1_median,DI_regenLight.2_mean,DI_regenLight.2_std,DI_regenLight.2_median,SteeringSpeed129_mean,SteeringSpeed129_std,SteeringSpeed129_median,DI_vehicleSpeed.1_mean,DI_vehicleSpeed.1_std,DI_vehicleSpeed.1_median
0,0.409091,0.088435,0.356873,0.213333,0.009524,1,0.736111,1,0.492665,0.835415,...,0.320000,0.100000,0.302532,0.0,0.185750,0.302687,0.007998,0.468630,0.346464,0.465535
1,0.238095,0.460870,0.274510,0.047619,0.658156,0,0.338672,0,0.116014,0.352682,...,0.047619,0.083333,0.278718,0.0,0.188079,0.321934,0.015872,0.467463,0.292303,0.436063
2,0.355932,0.000000,0.133337,0.031621,0.448980,0,0.864858,0,0.137157,0.823659,...,0.031621,0.300000,0.462125,0.0,0.201322,0.320330,0.015873,0.562356,0.315818,0.600904
3,0.083333,0.141667,0.047076,0.000000,0.765306,0,0.538007,0,0.061947,0.070536,...,0.023810,0.283333,0.454420,0.0,0.198197,0.327766,0.015934,0.506827,0.321552,0.504366
4,0.328358,0.000000,0.078523,0.000000,0.959183,1,0.277535,1,0.913379,0.753123,...,0.000000,0.516667,0.503939,1.0,0.125395,0.281336,0.000000,0.385537,0.270001,0.262644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46371,0.000000,0.000000,0.956862,0.049180,0.081634,1,0.261148,1,0.000000,0.000000,...,0.049180,0.500000,0.504219,0.5,0.166928,0.327417,0.000000,0.469956,0.278596,0.471809
46372,0.376147,0.000000,0.641778,0.841897,0.525510,1,0.862775,1,0.136502,0.749204,...,0.747036,0.316667,0.469102,0.0,0.226236,0.371422,0.007998,0.582130,0.295395,0.591398
46373,0.359375,0.113095,0.431392,0.000000,0.979591,1,0.537609,1,0.138981,0.855009,...,0.073171,0.200000,0.403376,0.0,0.185179,0.335577,0.015873,0.474120,0.320873,0.436888
46374,0.151515,0.432432,0.795276,0.900000,0.725490,0,0.995085,0,0.120952,0.082292,...,0.100000,0.316667,0.469102,0.0,0.202425,0.336263,0.015934,0.505695,0.295260,0.468184


In [11]:
X_valid = valid_data.drop('label', axis=1)
y_valid = valid_data['label']

In [12]:
X_valid

,RearHeatPower266,DI_accelPedalPos,BattVoltage132,VCFRONT_chillerExvFlowm3,BMS_maxDischargePower,DI_regenLight,DI_vehicleSpeed,DI_regenLight.1,DIR_torqueActual,RearTorque1D8,...,VCFRONT_chillerExvFlowm3.1_median,DI_regenLight.2_mean,DI_regenLight.2_std,DI_regenLight.2_median,SteeringSpeed129_mean,SteeringSpeed129_std,SteeringSpeed129_median,DI_vehicleSpeed.1_mean,DI_vehicleSpeed.1_std,DI_vehicleSpeed.1_median
0,0.275000,0.533333,0.728335,0.074074,0.076532,0,0.199845,0,0.094947,0.105805,...,0.055556,0.183333,0.390205,0.0,0.176766,0.311989,0.000062,0.474193,0.285063,0.434160
1,0.180000,0.198413,0.101975,0.073171,0.011364,0,0.400932,0,0.566121,0.929464,...,0.048780,0.166667,0.375823,0.0,0.239991,0.372777,0.023870,0.520129,0.294561,0.532246
2,0.084746,0.619403,0.572540,0.258065,0.010204,0,0.728822,0,0.138265,0.019593,...,0.258065,0.050000,0.219784,0.0,0.147070,0.270082,0.000000,0.536958,0.298795,0.602789
3,0.567164,0.040541,0.501991,0.056250,0.413274,1,0.666925,1,0.633269,0.723525,...,0.056250,0.333333,0.475383,0.0,0.307649,0.380890,0.063488,0.505728,0.314441,0.467934
4,0.090909,0.341085,0.141163,0.666667,0.789474,0,0.466494,0,0.066815,0.337007,...,0.666667,0.400000,0.494032,0.0,0.135240,0.284118,0.000062,0.519431,0.289234,0.533247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24769,0.101266,0.206452,0.925196,0.074074,0.433682,0,0.067976,0,0.709606,0.952976,...,0.074074,0.333333,0.475383,0.0,0.215917,0.323074,0.047678,0.426510,0.313758,0.398811
24770,0.211268,0.352564,0.152908,0.000000,0.000000,0,0.863812,0,0.001941,0.074455,...,0.022177,0.150000,0.360085,0.0,0.238404,0.347404,0.015965,0.519813,0.304084,0.531468
24771,0.000000,0.000000,0.278436,0.016393,0.086736,1,0.264805,1,0.000000,0.000000,...,0.024590,0.416667,0.497167,0.0,0.166399,0.330827,0.000000,0.396117,0.239205,0.264805
24772,0.648148,0.000000,0.745102,1.000000,0.055556,1,0.862087,1,0.207479,0.713936,...,0.000000,0.233333,0.426522,0.0,0.041794,0.150792,0.000000,0.426700,0.306310,0.334065


In [13]:
# 결측치 drop
X_train = X_train.dropna()
y_train = y_train[X_train.index]

X_valid = X_valid.dropna()
y_valid = y_valid[X_valid.index]

Mlp model training

In [14]:
# Multi-Layer Perceptron 모델
mlp_model = MLPClassifier(random_state=42)
mlp_model.fit(X_train, y_train)
y_pred_mlp = mlp_model.predict(X_valid)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [30]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_recall_fscore_support, precision_recall_curve
from sklearn.preprocessing import label_binarize
import numpy as np

# MLP 모델 훈련
mlp_model = MLPClassifier(random_state=42)
mlp_model.fit(X_train, y_train)
y_pred_mlp = mlp_model.predict(X_valid)

# 예측 확률 계산
y_pred_prob = mlp_model.predict_proba(X_valid)

# 레이블에 따라 확률을 그룹화
grouped_prob = {label: [] for label in np.unique(y_train)}
for prob, label in zip(y_pred_prob, y_valid):
    grouped_prob[label].append(prob)

# TOP-3 정확도 계산 함수
def top_3_accuracy(probs, labels):
    correct = 0
    for prob, label in zip(probs, labels):
        top_3_labels = np.argsort(prob)[-3:]
        correct += label in top_3_labels
    return correct / len(labels)

# 다중 클래스 MAP 계산 함수
def mean_average_precision_multiclass(probs, y_true, n_classes):
    y_true_bin = label_binarize(y_true, classes=range(n_classes))
    average_precisions = []
    for i in range(n_classes):
        precision, recall, _ = precision_recall_curve(y_true_bin[:, i], probs[:, i])
        ap = np.trapz(recall, precision)
        average_precisions.append(ap)
    return np.mean(average_precisions)

# F1 스코어 계산 함수
def f1_score(y_true, y_pred):
    _, _, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    return f1

# 예측을 집계하고 평가 지표를 계산하는 함수
def evaluate_performance(grouped_prob, vote_count, n_classes):
    y_true = []
    y_pred = []
    y_scores = []
    for label, probs in grouped_prob.items():
        for i in range(0, len(probs), vote_count):
            avg_prob = np.mean(probs[i:i + vote_count], axis=0)
            predicted_label = np.argmax(avg_prob)
            y_true.append(label)
            y_pred.append(predicted_label)
            y_scores.append(avg_prob)

    acc = np.mean(np.array(y_pred) == np.array(y_true))
    top_3_acc = top_3_accuracy(np.array(y_scores), np.array(y_true))
    map_score = mean_average_precision_multiclass(np.array(y_scores), np.array(y_true), n_classes)
    f1 = f1_score(np.array(y_true), np.array(y_pred))

    return acc, top_3_acc, map_score, f1

# 각 보팅 방식에 대한 평가 지표 계산
n_classes = len(np.unique(y_train))
for vote_count in [1, 3, 10, len(y_valid)]:
    acc, top_3_acc, map_score, f1 = evaluate_performance(grouped_prob, vote_count, n_classes)
    print(f"Voting {vote_count}: 정확도 = {acc}, TOP-3 정확도 = {top_3_acc}, MAP = {map_score}, F1 스코어 = {f1}")

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Voting 1: 정확도 = 0.24527213547031437, TOP-3 정확도 = 0.5488855012154423, MAP = 0.1321317000687465, F1 스코어 = 0.24102355044773435
Voting 3: 정확도 = 0.3112181863108475, TOP-3 정확도 = 0.603409933283914, MAP = 0.20570397458826284, F1 스코어 = 0.29835394459803327
Voting 10: 정확도 = 0.40131578947368424, TOP-3 정확도 = 0.7125822368421053, MAP = 0.3163934924570614, F1 스코어 = 0.3730054330606887
Voting 24271: 정확도 = 0.45454545454545453, TOP-3 정확도 = 0.8181818181818182, MAP = 0.6098484848484849, F1 스코어 = 0.4


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


코스별 교차검증

valid: B코스

In [31]:
df = pd.read_csv('./merge_data.csv', encoding='cp949')
df = df.sample(frac=1, random_state=7777).reset_index(drop=True)

df_train = df[df['course'] != 1]
df_valid = df[df['course'] == 1]

# 이후의 데이터 처리 코드
train_data = df_train.drop(['Unnamed: 0','course', 'round', 'timestamp_rounded'], axis=1)
valid_data = df_valid.drop(['Unnamed: 0','course', 'round', 'timestamp_rounded'], axis=1)

train_data.reset_index(inplace=True, drop=True)
valid_data.reset_index(inplace=True, drop=True)

X_train = train_data.drop('label', axis=1)
y_train = train_data['label']

X_valid = valid_data.drop('label', axis=1)
y_valid = valid_data['label']

# 결측치 drop
X_train = X_train.dropna()
y_train = y_train[X_train.index]

X_valid = X_valid.dropna()
y_valid = y_valid[X_valid.index]

In [32]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_recall_fscore_support, precision_recall_curve
from sklearn.preprocessing import label_binarize
import numpy as np

# MLP 모델 훈련
mlp_model = MLPClassifier(random_state=42)
mlp_model.fit(X_train, y_train)
y_pred_mlp = mlp_model.predict(X_valid)

# 예측 확률 계산
y_pred_prob = mlp_model.predict_proba(X_valid)

# 레이블에 따라 확률을 그룹화
grouped_prob = {label: [] for label in np.unique(y_train)}
for prob, label in zip(y_pred_prob, y_valid):
    grouped_prob[label].append(prob)

# TOP-3 정확도 계산 함수
def top_3_accuracy(probs, labels):
    correct = 0
    for prob, label in zip(probs, labels):
        top_3_labels = np.argsort(prob)[-3:]
        correct += label in top_3_labels
    return correct / len(labels)

# 다중 클래스 MAP 계산 함수
def mean_average_precision_multiclass(probs, y_true, n_classes):
    y_true_bin = label_binarize(y_true, classes=range(n_classes))
    average_precisions = []
    for i in range(n_classes):
        precision, recall, _ = precision_recall_curve(y_true_bin[:, i], probs[:, i])
        ap = np.trapz(recall, precision)
        average_precisions.append(ap)
    return np.mean(average_precisions)

# F1 스코어 계산 함수
def f1_score(y_true, y_pred):
    _, _, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    return f1

# 예측을 집계하고 평가 지표를 계산하는 함수
def evaluate_performance(grouped_prob, vote_count, n_classes):
    y_true = []
    y_pred = []
    y_scores = []
    for label, probs in grouped_prob.items():
        for i in range(0, len(probs), vote_count):
            avg_prob = np.mean(probs[i:i + vote_count], axis=0)
            predicted_label = np.argmax(avg_prob)
            y_true.append(label)
            y_pred.append(predicted_label)
            y_scores.append(avg_prob)

    acc = np.mean(np.array(y_pred) == np.array(y_true))
    top_3_acc = top_3_accuracy(np.array(y_scores), np.array(y_true))
    map_score = mean_average_precision_multiclass(np.array(y_scores), np.array(y_true), n_classes)
    f1 = f1_score(np.array(y_true), np.array(y_pred))

    return acc, top_3_acc, map_score, f1

# 각 보팅 방식에 대한 평가 지표 계산
n_classes = len(np.unique(y_train))
for vote_count in [1, 3, 10, len(y_valid)]:
    acc, top_3_acc, map_score, f1 = evaluate_performance(grouped_prob, vote_count, n_classes)
    print(f"Voting {vote_count}: 정확도 = {acc}, TOP-3 정확도 = {top_3_acc}, MAP = {map_score}, F1 스코어 = {f1}")

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Voting 1: 정확도 = 0.1977077363896848, TOP-3 정확도 = 0.45781598217128305, MAP = 0.09151699160844608, F1 스코어 = 0.1950745338479537
Voting 3: 정확도 = 0.24050391296048865, TOP-3 정확도 = 0.5218553158999809, MAP = 0.1411364854288405, F1 스코어 = 0.23199290064965153
Voting 10: 정확도 = 0.312579415501906, TOP-3 정확도 = 0.6372299872935197, MAP = 0.2304995669335551, F1 스코어 = 0.2929662121248659
Voting 15705: 정확도 = 0.45454545454545453, TOP-3 정확도 = 0.8181818181818182, MAP = 0.39664502164502163, F1 스코어 = 0.3696969696969697


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


코스별 교차검증

valid: C코스

In [33]:
df = pd.read_csv('./merge_data.csv', encoding='cp949')
df = df.sample(frac=1, random_state=7777).reset_index(drop=True)

df_train = df[df['course'] != 2]
df_valid = df[df['course'] == 2]

# 이후의 데이터 처리 코드
train_data = df_train.drop(['Unnamed: 0','course', 'round', 'timestamp_rounded'], axis=1)
valid_data = df_valid.drop(['Unnamed: 0','course', 'round', 'timestamp_rounded'], axis=1)

train_data.reset_index(inplace=True, drop=True)
valid_data.reset_index(inplace=True, drop=True)

X_train = train_data.drop('label', axis=1)
y_train = train_data['label']

X_valid = valid_data.drop('label', axis=1)
y_valid = valid_data['label']

# 결측치 drop
X_train = X_train.dropna()
y_train = y_train[X_train.index]

X_valid = X_valid.dropna()
y_valid = y_valid[X_valid.index]

In [34]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_recall_fscore_support, precision_recall_curve
from sklearn.preprocessing import label_binarize
import numpy as np

# MLP 모델 훈련
mlp_model = MLPClassifier(random_state=42)
mlp_model.fit(X_train, y_train)
y_pred_mlp = mlp_model.predict(X_valid)

# 예측 확률 계산
y_pred_prob = mlp_model.predict_proba(X_valid)

# 레이블에 따라 확률을 그룹화
grouped_prob = {label: [] for label in np.unique(y_train)}
for prob, label in zip(y_pred_prob, y_valid):
    grouped_prob[label].append(prob)

# TOP-3 정확도 계산 함수
def top_3_accuracy(probs, labels):
    correct = 0
    for prob, label in zip(probs, labels):
        top_3_labels = np.argsort(prob)[-3:]
        correct += label in top_3_labels
    return correct / len(labels)

# 다중 클래스 MAP 계산 함수
def mean_average_precision_multiclass(probs, y_true, n_classes):
    y_true_bin = label_binarize(y_true, classes=range(n_classes))
    average_precisions = []
    for i in range(n_classes):
        precision, recall, _ = precision_recall_curve(y_true_bin[:, i], probs[:, i])
        ap = np.trapz(recall, precision)
        average_precisions.append(ap)
    return np.mean(average_precisions)

# F1 스코어 계산 함수
def f1_score(y_true, y_pred):
    _, _, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    return f1

# 예측을 집계하고 평가 지표를 계산하는 함수
def evaluate_performance(grouped_prob, vote_count, n_classes):
    y_true = []
    y_pred = []
    y_scores = []
    for label, probs in grouped_prob.items():
        for i in range(0, len(probs), vote_count):
            avg_prob = np.mean(probs[i:i + vote_count], axis=0)
            predicted_label = np.argmax(avg_prob)
            y_true.append(label)
            y_pred.append(predicted_label)
            y_scores.append(avg_prob)

    acc = np.mean(np.array(y_pred) == np.array(y_true))
    top_3_acc = top_3_accuracy(np.array(y_scores), np.array(y_true))
    map_score = mean_average_precision_multiclass(np.array(y_scores), np.array(y_true), n_classes)
    f1 = f1_score(np.array(y_true), np.array(y_pred))

    return acc, top_3_acc, map_score, f1

# 각 보팅 방식에 대한 평가 지표 계산
n_classes = len(np.unique(y_train))
for vote_count in [1, 3, 10, len(y_valid)]:
    acc, top_3_acc, map_score, f1 = evaluate_performance(grouped_prob, vote_count, n_classes)
    print(f"Voting {vote_count}: 정확도 = {acc}, TOP-3 정확도 = {top_3_acc}, MAP = {map_score}, F1 스코어 = {f1}")

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Voting 1: 정확도 = 0.21365321252059308, TOP-3 정확도 = 0.5101592531576057, MAP = 0.12116621766414512, F1 스코어 = 0.2091440649436155
Voting 3: 정확도 = 0.2557636887608069, TOP-3 정확도 = 0.526039522437217, MAP = 0.18991097547647195, F1 스코어 = 0.2416113264314317
Voting 10: 정확도 = 0.3127783487495718, TOP-3 정확도 = 0.5943816375471052, MAP = 0.26879473718160746, F1 스코어 = 0.2863563630530212
Voting 29136: 정확도 = 0.36363636363636365, TOP-3 정확도 = 0.8181818181818182, MAP = 0.3170995670995671, F1 스코어 = 0.303030303030303


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


회차별 교차검증

valid: 1회차

In [35]:
df = pd.read_csv('./merge_data.csv', encoding='cp949')
df = df.sample(frac=1, random_state=7777).reset_index(drop=True)

df_train = df[df['round'] != 1]
df_valid = df[df['round'] == 1]

# 이후의 데이터 처리 코드
train_data = df_train.drop(['Unnamed: 0','course', 'round', 'timestamp_rounded'], axis=1)
valid_data = df_valid.drop(['Unnamed: 0','course', 'round', 'timestamp_rounded'], axis=1)

train_data.reset_index(inplace=True, drop=True)
valid_data.reset_index(inplace=True, drop=True)

X_train = train_data.drop('label', axis=1)
y_train = train_data['label']

X_valid = valid_data.drop('label', axis=1)
y_valid = valid_data['label']

# 결측치 drop
X_train = X_train.dropna()
y_train = y_train[X_train.index]

X_valid = X_valid.dropna()
y_valid = y_valid[X_valid.index]

In [36]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_recall_fscore_support, precision_recall_curve
from sklearn.preprocessing import label_binarize
import numpy as np

# MLP 모델 훈련
mlp_model = MLPClassifier(random_state=42)
mlp_model.fit(X_train, y_train)
y_pred_mlp = mlp_model.predict(X_valid)

# 예측 확률 계산
y_pred_prob = mlp_model.predict_proba(X_valid)

# 레이블에 따라 확률을 그룹화
grouped_prob = {label: [] for label in np.unique(y_train)}
for prob, label in zip(y_pred_prob, y_valid):
    grouped_prob[label].append(prob)

# TOP-3 정확도 계산 함수
def top_3_accuracy(probs, labels):
    correct = 0
    for prob, label in zip(probs, labels):
        top_3_labels = np.argsort(prob)[-3:]
        correct += label in top_3_labels
    return correct / len(labels)

# 다중 클래스 MAP 계산 함수
def mean_average_precision_multiclass(probs, y_true, n_classes):
    y_true_bin = label_binarize(y_true, classes=range(n_classes))
    average_precisions = []
    for i in range(n_classes):
        precision, recall, _ = precision_recall_curve(y_true_bin[:, i], probs[:, i])
        ap = np.trapz(recall, precision)
        average_precisions.append(ap)
    return np.mean(average_precisions)

# F1 스코어 계산 함수
def f1_score(y_true, y_pred):
    _, _, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    return f1

# 예측을 집계하고 평가 지표를 계산하는 함수
def evaluate_performance(grouped_prob, vote_count, n_classes):
    y_true = []
    y_pred = []
    y_scores = []
    for label, probs in grouped_prob.items():
        for i in range(0, len(probs), vote_count):
            avg_prob = np.mean(probs[i:i + vote_count], axis=0)
            predicted_label = np.argmax(avg_prob)
            y_true.append(label)
            y_pred.append(predicted_label)
            y_scores.append(avg_prob)

    acc = np.mean(np.array(y_pred) == np.array(y_true))
    top_3_acc = top_3_accuracy(np.array(y_scores), np.array(y_true))
    map_score = mean_average_precision_multiclass(np.array(y_scores), np.array(y_true), n_classes)
    f1 = f1_score(np.array(y_true), np.array(y_pred))

    return acc, top_3_acc, map_score, f1

# 각 보팅 방식에 대한 평가 지표 계산
n_classes = len(np.unique(y_train))
for vote_count in [1, 3, 10, len(y_valid)]:
    acc, top_3_acc, map_score, f1 = evaluate_performance(grouped_prob, vote_count, n_classes)
    print(f"Voting {vote_count}: 정확도 = {acc}, TOP-3 정확도 = {top_3_acc}, MAP = {map_score}, F1 스코어 = {f1}")

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Voting 1: 정확도 = 0.25758337523043406, TOP-3 정확도 = 0.5470644098095079, MAP = 0.1396052165632258, F1 스코어 = 0.24853353575355633
Voting 3: 정확도 = 0.340536013400335, TOP-3 정확도 = 0.606532663316583, MAP = 0.22429022122404182, F1 스코어 = 0.314958093582115
Voting 10: 정확도 = 0.45712694877505566, TOP-3 정확도 = 0.7021158129175946, MAP = 0.36628883476383844, F1 스코어 = 0.40539443712697326
Voting 17901: 정확도 = 0.6363636363636364, TOP-3 정확도 = 0.8181818181818182, MAP = 0.6704545454545454, F1 스코어 = 0.5303030303030304


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


회차별 교차검증

valid: 2회차

In [4]:
df = pd.read_csv('./merge_data.csv', encoding='cp949')
df = df.sample(frac=1, random_state=7777).reset_index(drop=True)

df_train = df[df['round'] != 2]
df_valid = df[df['round'] == 2]

# 이후의 데이터 처리 코드
train_data = df_train.drop(['Unnamed: 0','course', 'round', 'timestamp_rounded'], axis=1)
valid_data = df_valid.drop(['Unnamed: 0','course', 'round', 'timestamp_rounded'], axis=1)

train_data.reset_index(inplace=True, drop=True)
valid_data.reset_index(inplace=True, drop=True)

X_train = train_data.drop('label', axis=1)
y_train = train_data['label']

X_valid = valid_data.drop('label', axis=1)
y_valid = valid_data['label']

# 결측치 drop
X_train = X_train.dropna()
y_train = y_train[X_train.index]

X_valid = X_valid.dropna()
y_valid = y_valid[X_valid.index]

In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_recall_fscore_support, precision_recall_curve
from sklearn.preprocessing import label_binarize
import numpy as np

# MLP 모델 훈련
mlp_model = MLPClassifier(random_state=42)
mlp_model.fit(X_train, y_train)
y_pred_mlp = mlp_model.predict(X_valid)

# 예측 확률 계산
y_pred_prob = mlp_model.predict_proba(X_valid)

# 레이블에 따라 확률을 그룹화
grouped_prob = {label: [] for label in np.unique(y_train)}
for prob, label in zip(y_pred_prob, y_valid):
    grouped_prob[label].append(prob)

# TOP-3 정확도 계산 함수
def top_3_accuracy(probs, labels):
    correct = 0
    for prob, label in zip(probs, labels):
        top_3_labels = np.argsort(prob)[-3:]
        correct += label in top_3_labels
    return correct / len(labels)

# 다중 클래스 MAP 계산 함수
def mean_average_precision_multiclass(probs, y_true, n_classes):
    y_true_bin = label_binarize(y_true, classes=range(n_classes))
    average_precisions = []
    for i in range(n_classes):
        precision, recall, _ = precision_recall_curve(y_true_bin[:, i], probs[:, i])
        ap = np.trapz(recall, precision)
        average_precisions.append(ap)
    return np.mean(average_precisions)

# F1 스코어 계산 함수
def f1_score(y_true, y_pred):
    _, _, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    return f1

# 예측을 집계하고 평가 지표를 계산하는 함수
def evaluate_performance(grouped_prob, vote_count, n_classes):
    y_true = []
    y_pred = []
    y_scores = []
    for label, probs in grouped_prob.items():
        for i in range(0, len(probs), vote_count):
            avg_prob = np.mean(probs[i:i + vote_count], axis=0)
            predicted_label = np.argmax(avg_prob)
            y_true.append(label)
            y_pred.append(predicted_label)
            y_scores.append(avg_prob)

    acc = np.mean(np.array(y_pred) == np.array(y_true))
    top_3_acc = top_3_accuracy(np.array(y_scores), np.array(y_true))
    map_score = mean_average_precision_multiclass(np.array(y_scores), np.array(y_true), n_classes)
    f1 = f1_score(np.array(y_true), np.array(y_pred))

    return acc, top_3_acc, map_score, f1

# 각 보팅 방식에 대한 평가 지표 계산
n_classes = len(np.unique(y_train))
for vote_count in [1, 3, 10, len(y_valid)]:
    acc, top_3_acc, map_score, f1 = evaluate_performance(grouped_prob, vote_count, n_classes)
    print(f"Voting {vote_count}: 정확도 = {acc}, TOP-3 정확도 = {top_3_acc}, MAP = {map_score}, F1 스코어 = {f1}")

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Voting 1: 정확도 = 0.2963954796208023, TOP-3 정확도 = 0.5765090246906968, MAP = 0.20356641522078578, F1 스코어 = 0.30576273507142593
Voting 3: 정확도 = 0.3874076453581754, TOP-3 정확도 = 0.6885640860905878, MAP = 0.31718301239542696, F1 스코어 = 0.3920911361116082
Voting 10: 정확도 = 0.5146823278163374, TOP-3 정확도 = 0.8088627869727709, MAP = 0.45116560037382114, F1 스코어 = 0.5110297968881274
Voting 18671: 정확도 = 0.7272727272727273, TOP-3 정확도 = 1.0, MAP = 0.6363636363636364, F1 스코어 = 0.6818181818181818


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


회차별 교차검증

valid: 3회차

In [6]:
df = pd.read_csv('./merge_data.csv', encoding='cp949')
df = df.sample(frac=1, random_state=7777).reset_index(drop=True)

df_train = df[df['round'] != 3]
df_valid = df[df['round'] == 3]

# 이후의 데이터 처리 코드
train_data = df_train.drop(['Unnamed: 0','course', 'round', 'timestamp_rounded'], axis=1)
valid_data = df_valid.drop(['Unnamed: 0','course', 'round', 'timestamp_rounded'], axis=1)

train_data.reset_index(inplace=True, drop=True)
valid_data.reset_index(inplace=True, drop=True)

X_train = train_data.drop('label', axis=1)
y_train = train_data['label']

X_valid = valid_data.drop('label', axis=1)
y_valid = valid_data['label']

# 결측치 drop
X_train = X_train.dropna()
y_train = y_train[X_train.index]

X_valid = X_valid.dropna()
y_valid = y_valid[X_valid.index]

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_recall_fscore_support, precision_recall_curve
from sklearn.preprocessing import label_binarize
import numpy as np

# MLP 모델 훈련
mlp_model = MLPClassifier(random_state=42)
mlp_model.fit(X_train, y_train)
y_pred_mlp = mlp_model.predict(X_valid)

# 예측 확률 계산
y_pred_prob = mlp_model.predict_proba(X_valid)

# 레이블에 따라 확률을 그룹화
grouped_prob = {label: [] for label in np.unique(y_train)}
for prob, label in zip(y_pred_prob, y_valid):
    grouped_prob[label].append(prob)

# TOP-3 정확도 계산 함수
def top_3_accuracy(probs, labels):
    correct = 0
    for prob, label in zip(probs, labels):
        top_3_labels = np.argsort(prob)[-3:]
        correct += label in top_3_labels
    return correct / len(labels)

# 다중 클래스 MAP 계산 함수
def mean_average_precision_multiclass(probs, y_true, n_classes):
    y_true_bin = label_binarize(y_true, classes=range(n_classes))
    average_precisions = []
    for i in range(n_classes):
        precision, recall, _ = precision_recall_curve(y_true_bin[:, i], probs[:, i])
        ap = np.trapz(recall, precision)
        average_precisions.append(ap)
    return np.mean(average_precisions)

# F1 스코어 계산 함수
def f1_score(y_true, y_pred):
    _, _, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    return f1

# 예측을 집계하고 평가 지표를 계산하는 함수
def evaluate_performance(grouped_prob, vote_count, n_classes):
    y_true = []
    y_pred = []
    y_scores = []
    for label, probs in grouped_prob.items():
        for i in range(0, len(probs), vote_count):
            avg_prob = np.mean(probs[i:i + vote_count], axis=0)
            predicted_label = np.argmax(avg_prob)
            y_true.append(label)
            y_pred.append(predicted_label)
            y_scores.append(avg_prob)

    acc = np.mean(np.array(y_pred) == np.array(y_true))
    top_3_acc = top_3_accuracy(np.array(y_scores), np.array(y_true))
    map_score = mean_average_precision_multiclass(np.array(y_scores), np.array(y_true), n_classes)
    f1 = f1_score(np.array(y_true), np.array(y_pred))

    return acc, top_3_acc, map_score, f1

# 각 보팅 방식에 대한 평가 지표 계산
n_classes = len(np.unique(y_train))
for vote_count in [1, 3, 10, len(y_valid)]:
    acc, top_3_acc, map_score, f1 = evaluate_performance(grouped_prob, vote_count, n_classes)
    print(f"Voting {vote_count}: 정확도 = {acc}, TOP-3 정확도 = {top_3_acc}, MAP = {map_score}, F1 스코어 = {f1}")

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Voting 1: 정확도 = 0.25573501872659177, TOP-3 정확도 = 0.568878745318352, MAP = 0.16055868521511232, F1 스코어 = 0.2526009702995983
Voting 3: 정확도 = 0.3172486401123004, TOP-3 정확도 = 0.6232672398666432, MAP = 0.22838783706636276, F1 스코어 = 0.3063827596527639
Voting 10: 정확도 = 0.36443148688046645, TOP-3 정확도 = 0.717201166180758, MAP = 0.3154952817939876, F1 스코어 = 0.35400312193306105
Voting 17088: 정확도 = 0.45454545454545453, TOP-3 정확도 = 0.8181818181818182, MAP = 0.47878787878787876, F1 스코어 = 0.4242424242424242


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


회차별 교차검증

valid: 4회차

In [8]:
df = pd.read_csv('./merge_data.csv', encoding='cp949')
df = df.sample(frac=1, random_state=7777).reset_index(drop=True)

df_train = df[df['round'] != 4]
df_valid = df[df['round'] == 4]

# 이후의 데이터 처리 코드
train_data = df_train.drop(['Unnamed: 0','course', 'round', 'timestamp_rounded'], axis=1)
valid_data = df_valid.drop(['Unnamed: 0','course', 'round', 'timestamp_rounded'], axis=1)

train_data.reset_index(inplace=True, drop=True)
valid_data.reset_index(inplace=True, drop=True)

X_train = train_data.drop('label', axis=1)
y_train = train_data['label']

X_valid = valid_data.drop('label', axis=1)
y_valid = valid_data['label']

# 결측치 drop
X_train = X_train.dropna()
y_train = y_train[X_train.index]

X_valid = X_valid.dropna()
y_valid = y_valid[X_valid.index]

In [9]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_recall_fscore_support, precision_recall_curve
from sklearn.preprocessing import label_binarize
import numpy as np

# MLP 모델 훈련
mlp_model = MLPClassifier(random_state=42)
mlp_model.fit(X_train, y_train)
y_pred_mlp = mlp_model.predict(X_valid)

# 예측 확률 계산
y_pred_prob = mlp_model.predict_proba(X_valid)

# 레이블에 따라 확률을 그룹화
grouped_prob = {label: [] for label in np.unique(y_train)}
for prob, label in zip(y_pred_prob, y_valid):
    grouped_prob[label].append(prob)

# TOP-3 정확도 계산 함수
def top_3_accuracy(probs, labels):
    correct = 0
    for prob, label in zip(probs, labels):
        top_3_labels = np.argsort(prob)[-3:]
        correct += label in top_3_labels
    return correct / len(labels)

# 다중 클래스 MAP 계산 함수
def mean_average_precision_multiclass(probs, y_true, n_classes):
    y_true_bin = label_binarize(y_true, classes=range(n_classes))
    average_precisions = []
    for i in range(n_classes):
        precision, recall, _ = precision_recall_curve(y_true_bin[:, i], probs[:, i])
        ap = np.trapz(recall, precision)
        average_precisions.append(ap)
    return np.mean(average_precisions)

# F1 스코어 계산 함수
def f1_score(y_true, y_pred):
    _, _, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    return f1

# 예측을 집계하고 평가 지표를 계산하는 함수
def evaluate_performance(grouped_prob, vote_count, n_classes):
    y_true = []
    y_pred = []
    y_scores = []
    for label, probs in grouped_prob.items():
        for i in range(0, len(probs), vote_count):
            avg_prob = np.mean(probs[i:i + vote_count], axis=0)
            predicted_label = np.argmax(avg_prob)
            y_true.append(label)
            y_pred.append(predicted_label)
            y_scores.append(avg_prob)

    acc = np.mean(np.array(y_pred) == np.array(y_true))
    top_3_acc = top_3_accuracy(np.array(y_scores), np.array(y_true))
    map_score = mean_average_precision_multiclass(np.array(y_scores), np.array(y_true), n_classes)
    f1 = f1_score(np.array(y_true), np.array(y_pred))

    return acc, top_3_acc, map_score, f1

# 각 보팅 방식에 대한 평가 지표 계산
n_classes = len(np.unique(y_train))
for vote_count in [1, 3, 10, len(y_valid)]:
    acc, top_3_acc, map_score, f1 = evaluate_performance(grouped_prob, vote_count, n_classes)
    print(f"Voting {vote_count}: 정확도 = {acc}, TOP-3 정확도 = {top_3_acc}, MAP = {map_score}, F1 스코어 = {f1}")

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Voting 1: 정확도 = 0.24236344809733368, TOP-3 정확도 = 0.5457545948744499, MAP = 0.12479937902559879, F1 스코어 = 0.22986606525456799
Voting 3: 정확도 = 0.301454898157129, TOP-3 정확도 = 0.5807953443258972, MAP = 0.21402905150724694, F1 스코어 = 0.27523034734742563
Voting 10: 정확도 = 0.3836234687298517, TOP-3 정확도 = 0.6802063185041909, MAP = 0.33595596954509077, F1 스코어 = 0.3462683043273056
Voting 15452: 정확도 = 0.5454545454545454, TOP-3 정확도 = 0.8181818181818182, MAP = 0.5965909090909091, F1 스코어 = 0.5151515151515151


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
